In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import get_raw_data
import get_preprocessing_data
import analysis_method
import statsmodels.regression.linear_model as sm

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pv_list = ["0102", "1701", "1702", "0111", "0330", "0331", "0332", "0333", "0101", "0107", "0110", "5501"]
#["0102"(가구주와의 관계), "1701"(근로소득 유무), "1702"(총 연간근로소득), "0111"(학력졸업여부), "0330"(업종8차),
# "0331"(업종9차), "0332"(직종8차), "0333"(직종9차), "0101"(성별), "0107"(만나이), "0110"(학력학교), "5501"(혼인상태)]
hv_list = ["0150", "1406", "2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155",
           "2156", "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
           "2189", "2190", "2191"]
income_code_list = ["2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155", "2156",
                    "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
                    "2189", "2190", "2191"]

cpi_data = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls").loc[0, :]
cpi_data_index = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls", dtype="str").loc[14, :].values[1:]
cpi_data = pd.Series(cpi_data.values[1:], dtype="float")
cpi_data.name = "CPI"
cpi_data.index = cpi_data_index

industry_8 = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="8차")
industry_large = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="대분류")

industry_return = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\oversea_return(5_index_without_nasdaq).xlsx", index_col=0)

data_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]

cpi_multiple = get_preprocessing_data.get_cpi_multi(cpi_data, data_num)

data_dict = {}
for i in data_num:
    h_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\h_data\\h_" + i + ".pickle"
    p_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\p_data\\p_" + i + ".pickle"

    h_data = pd.read_pickle(h_path)
    p_data = pd.read_pickle(p_path)

    raw_hdata = get_raw_data.get_hv_data(h_data, hv_list, i)
    raw_pdata = get_raw_data.get_pv_data(p_data, pv_list, i)
    
    fin_p_data = get_preprocessing_data.get_fin_personal(raw_pdata, industry_8, industry_large)
    income_data = get_preprocessing_data.get_income_data(fin_p_data.index, raw_hdata)

    fin_x_value = get_preprocessing_data.get_final_xvalue(raw_hdata, fin_p_data)
    fin_y_value = get_preprocessing_data.get_final_yvalue(income_data, cpi_multiple[i])

    finally_preprocessed_data = get_preprocessing_data.get_total_data(fin_y_value, fin_x_value)

    data_dict[i] = finally_preprocessed_data

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_df = pd.DataFrame()
for i in data_dict.keys():
    data_df = pd.concat([data_df, data_dict[i]])

_______________________________


### log(Y*)의 계산

In [3]:
temp_edu = data_df.groupby(["0110", "0330"]).get_group((1, "제조업 (10 ~ 33)"))
temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, 22)
x_mat = analysis_method.get_reg_x(temp_edu).iloc[:, :3]
fer_results = analysis_method.fe_regression_sol(temp_edu, False)

poly_x_mat = analysis_method.polynomial_matrix(temp_edu["0107"], 3) .iloc[:, 1:]
poly_x_mat.index = x_mat.index
fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

In [4]:
fer_results[0][:3]

intercept     7.671403
marriage      0.216859
family_num    0.068147
dtype: float64

In [5]:
fitting_result[1]

intercept    -0.310767
age^1/10^0   -0.027853
age^2/10^1    0.015535
age^3/10^2   -0.001434
dtype: float64

In [6]:
fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
fitting_paramters

intercept     7.360636
marriage      0.216859
family_num    0.068147
age^1/10^0   -0.027853
age^2/10^1    0.015535
age^3/10^2   -0.001434
dtype: float64

In [7]:
real_data = pd.concat([x_mat, poly_x_mat], axis=1)
real_data

,intercept,marriage,family_num,age^1/10^0,age^2/10^1,age^3/10^2
hhid,,,,,,
91.0,1,1.0,5.0,51.0,260.1,1326.51
99.0,1,1.0,4.0,51.0,260.1,1326.51
115.0,1,1.0,4.0,43.0,184.9,795.07
173.0,1,1.0,4.0,38.0,144.4,548.72
175.0,1,1.0,4.0,42.0,176.4,740.88
...,...,...,...,...,...,...
110251.0,1,0.0,1.0,24.0,57.6,138.24
101398.0,1,1.0,3.0,61.0,372.1,2269.81
110252.0,1,0.0,1.0,24.0,57.6,138.24


In [8]:
temp_edu["labor income"]

hhid
91.0        1940.968781
99.0        3105.550050
115.0       4852.421952
173.0       3881.937562
175.0       3881.937562
               ...     
110251.0    2410.000000
101398.0    4360.000000
110252.0    2760.000000
110085.0    6140.000000
110102.0    3500.000000
Name: labor income, Length: 6004, dtype: float64

In [9]:
real_data @ fitting_paramters

hhid
91.0        8.636426
99.0        8.568280
115.0       8.384863
173.0       8.248181
175.0       8.358367
              ...   
110251.0    7.456930
101398.0    8.608988
110252.0    7.456930
110085.0    8.163537
110102.0    7.889237
Length: 6004, dtype: float64

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 한번 추정하여 rid계산하는 경우)

In [10]:
fitting_paramters

intercept     7.360636
marriage      0.216859
family_num    0.068147
age^1/10^0   -0.027853
age^2/10^1    0.015535
age^3/10^2   -0.001434
dtype: float64

----------------------------------------
### shock calculation

In [11]:
industry_list = data_df["0330"].value_counts()[:15].index
industry_list

Index(['제조업 (10 ~ 33)', '도매 및 소매업 (45~47)', '건설업 (41 ~ 42)', '운수업(49~52)',
       '교육 서비스업(85)', '협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)',
       '공공행정, 국방 및 사회보장 행정(84)', '출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)',
       '금융 및 보험업 (64 ~ 66)', '농업, 임업 및 어업 (01 ~ 03)', '부동산업 및 임대업 (68 ~ 69)',
       '숙박 및 음식점업 (55 ~ 56)', '하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39)',
       '전문, 과학 및 기술 서비스업 (70 ~ 73)', '보건업 및 사회복지 서비스업(86~87)'],
      dtype='object')

In [12]:
group_sigma_dict = {} 
group_rid_dict = {}
for l in industry_list:
    group_sigma_df = pd.DataFrame()
    group_pvalue_df = pd.DataFrame()
    group_rid_df = pd.DataFrame()

    start_list = [22, 24, 26]
    for i in range(data_df["0110"].max()):
        df_y_star = pd.DataFrame()

        temp_edu = data_df.groupby(["0110", "0330"]).get_group((i+1, l))
        temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, start_list[i])

        fer_results = analysis_method.fe_regression_sol(temp_edu, False)
        fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

        fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)

        for j in data_num:
            temp_edu_split = data_dict[j].groupby(["0110", "0330"]).get_group((i+1, l))
            temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])

            log_y = np.log(temp_edu_split["labor income"])

            x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
            poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
            poly_x_mat.index = x_mat.index

            temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)

            f_hat = temp_real_data @ fitting_paramters

            log_y_star = log_y - f_hat
            df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

        df_y_star.columns = data_num

        temp_varrid_list = []
        temp_rid_list = []
        for k in range(len(data_num)-1):
            temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
            temp_rid_list.append(temp_rid)

            temp_varrid_list.append(np.nanvar(temp_rid))

        var_rid_series = pd.Series(temp_varrid_list)

        rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
        group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)

        var_rid_x = np.ones([len(var_rid_series), 2]) * 2
        var_rid_x[:, 0] = var_rid_series.index.values + 1
        var_rid_y = var_rid_series.values

        sol = sm.OLS(var_rid_y, var_rid_x)
        sol_result = sol.fit()

        sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
        pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])

        group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
        group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

    group_rid_df.columns = ["high_school", "college", "university"]
    group_sigma_df.columns = ["high_school", "college", "university"]
    group_pvalue_df.columns = ["high_school", "college", "university"]
    group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
    group_sigma_dict[l] = group_sigma_df
    group_rid_dict[l] = group_rid_df

C:\Users\JeayongYu\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: Degrees of freedom <= 0 for slice.
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\JeayongYu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [13]:
pd.DataFrame.from_dict({(i,j): group_sigma_dict[i][j]
                        for i in group_sigma_dict.keys()
                        for j in group_sigma_dict[i].columns}, orient="index")

idiosyncratic_temporary_shock  \
제조업 (10 ~ 33)                        high_school                       0.128567   
                                     college                           0.132352   
                                     university                        0.098062   
도매 및 소매업 (45~47)                     high_school                       0.230334   
                                     college                           0.152413   
                                     university                        0.180441   
건설업 (41 ~ 42)                        high_school                       0.161094   
                                     college                           0.026122   
                                     university                        0.105717   
운수업(49~52)                           high_school                       0.110736   
                                     college                           0.104665   
                                     university                        0.102564   
교육 서비스업(85)                          high_school                       0.165170   
                                     college                                NaN   
                                     university                        0.097606   
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     high_school                       0.205737   
                                     college                           0.097313   
                                     university                        0.210729   
공공행정, 국방 및 사회보장 행정(84)               high_school                       0.063387   
                                     college                           0.080998   
                                     university                        0.130281   
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     high_school                       0.176051   
                                     college                           0.232318   
                                     university                        0.156717   
금융 및 보험업 (64 ~ 66)                   high_school                       0.291905   
                                     college                           0.047781   
                                     university                        0.088360   
농업, 임업 및 어업 (01 ~ 03)                high_school                       0.326568   
                                     college                           0.719001   
                                     university                        0.286442   
부동산업 및 임대업 (68 ~ 69)                 high_school                       0.319312   
                                     college                           0.142916   
                                     university                             NaN   
숙박 및 음식점업 (55 ~ 56)                  high_school                       0.270627   
                                     college                           0.275648   
                                     university                        0.417059   
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) high_school                       0.148473   
                                     college                                NaN   
                                     university                        0.149371   
전문, 과학 및 기술 서비스업 (70 ~ 73)           high_school                       0.334095   
                                     college                                NaN   
                                     university                        0.211715   
보건업 및 사회복지 서비스업(86~87)               high_school                            NaN   
                                     college                                NaN   
                                     university                        0.122719   

                                                  permanent_shock  \
제조업 (10 ~ 33)                        high_school         0.005564   
                                     college             0.004175   
                        

----------------------------------------
### Beta & Correlation

In [14]:
ksi = group_rid_dict
ksi = pd.DataFrame.from_dict({(i,j): ksi[i][j]
                        for i in ksi.keys()
                        for j in ksi[i].columns}, orient="columns")
ksi

제조업 (10 ~ 33)                      도매 및 소매업 (45~47)                       \
     high_school   college university      high_school   college university   
02      0.007936 -0.015213  -0.168354         0.047998 -0.216268   0.005373   
03      0.121108  0.109246   0.102576         0.037752  0.107304   0.070973   
04     -0.023950  0.056576   0.091451         0.009540 -0.170712   0.329930   
05      0.122707  0.274583   0.085298         0.170987  0.363985   0.159425   
06      0.020656 -0.052918   0.032888         0.030831  0.033840  -0.027450   
07      0.074780  0.131687   0.041172        -0.016062  0.087894  -0.042301   
08      0.083470  0.099782   0.080112         0.086254  0.050811   0.146413   
09      0.047586  0.082750   0.043073         0.120813  0.091605   0.049105   
10      0.078421  0.250875   0.031349        -0.042151  0.096209   0.155673   
11      0.059346  0.094015  -0.009247         0.067231 -0.016178   0.027143   
12     -0.058674 -0.055999  -0.036908        -0.063784 -0.008752   0.034829   
13      0.018799  0.007880   0.031020         0.134383 -0.020572   0.095717   
14      0.075154  0.014054   0.122203         0.121494  0.007894  -0.069367   
15      0.030897  0.035795   0.028640        -0.070758  0.042744   0.040489   
16      0.044527  0.037914   0.067321         0.058498 -0.028287   0.005821   
17      0.026370  0.030604   0.009034         0.037838  0.057611  -0.048287   
18      0.014769 -0.002349   0.001254         0.029775  0.057380   0.035751   
19      0.044185  0.045973  -0.005797         0.040638  0.023579  -0.024350   
20      0.006813  0.057697  -0.017589        -0.009437 -0.014199   0.081272   
21      0.028423 -0.052852   0.037628         0.043337 -0.045669   0.020934   

   건설업 (41 ~ 42)                       운수업(49~52)  ... 숙박 및 음식점업 (55 ~ 56)  \
     high_school   college university high_school  ...          university   
02     -0.010032  0.089944  -0.111470    0.040137  ...           -0.113438   
03      0.159192  0.145865  -0.030692   -0.022948  ...           -0.055891   
04     -0.040508 -0.018140   0.034383   -0.046301  ...            0.169800   
05      0.340476  0.113331   0.055930    0.190651  ...            0.013178   
06      0.081855  0.067354   0.143102    0.059257  ...            0.169455   
07      0.060136  0.063546   0.108684    0.088376  ...           -0.050348   
08     -0.017736  0.460501   0.156956    0.005948  ...           -0.076455   
09      0.152887  0.234690   0.053747    0.074090  ...            0.323796   
10      0.019184  0.107719  -0.018915    0.068813  ...            0.325086   
11      0.015109  0.063075   0.092624    0.013874  ...            0.213155   
12     -0.024735 -0.024850  -0.024349   -0.075282  ...           -0.261137   
13      0.130247 -0.000034   0.183821    0.116805  ...            0.137881   
14      0.054316 -0.051046   0.109588    0.041827  ...           -0.008953   
15      0.112956  0.036584   0.013071    0.060065  ...           -0.013589   
16      0.052958  0.006262  -0.017650    0.004752  ...            0.003322   
17      0.103464 -0.000543   0.020792    0.061517  ...            0.002578   
18      0.022046 -0.003612   0.007192    0.016380  ...           -0.106549   
19      0.069472  0.018347   0.032737    0.023663  ...           -0.091195   
20      0.050565  0.081795  -0.020756    0.140234  ...            0.284360   
21      0.010901  0.021683   0.034308    0.031035  ...           -0.009280   

   하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39)                       \
                            high_school   college university   
02                            -0.114693 -0.205260   0.151646   
03                             0.345524  0.054593  -0.080510   
04                            -0.081719  0.176880   0.217866   
05                            -0.276414 -0.092816  -0.172874   
06                             0.042524  0.164031   0.194906   
07                             0.336040  0.420288  -0.062724   
08                            -0.00

In [15]:
industry_return.index = data_num
industry_ret = industry_return.copy() / 100

beta_dict = {}
#pvalue_dict = {}
for j in group_rid_dict.keys():
    beta_dict_l2 = {}
    for i in group_rid_dict[j].columns:
        beta_list = []
        pvalue_list = []
        eta_list = []
        
        temp_excess_ret = (industry_ret - 0.02)
        for k in industry_ret.columns:
            temp_eta = temp_excess_ret[k] - np.mean(temp_excess_ret)[k]
            eta_list.append(temp_eta)
            
            temp_beta_model = sm.OLS(ksi[j][i], temp_eta.loc[ksi[j].index])
            temp_beta_results = temp_beta_model.fit()
            
            beta_list.append(temp_beta_results.params[0])
            pvalue_list.append(temp_beta_results.pvalues[0])
        
        eta = pd.DataFrame(eta_list, index=industry_ret.columns).T
        beta = pd.DataFrame(beta_list, index=industry_ret.columns, columns=["Beta"])
        pvalue = pd.DataFrame(pvalue_list, index=industry_ret.columns, columns=["p-value"])
    
        beta_dict_l2[i] = pd.concat([beta, pvalue], axis=1).T
    beta_dict[j] = beta_dict_l2
    #pvalue_dict[j] = pvalue

In [16]:
pd.DataFrame.from_dict({(i,j,k): beta_dict[i][j][k]
                        for i in beta_dict.keys()
                        for j in beta_dict[i].keys()
                        for k in beta_dict[i][j].columns}, orient="index")

Beta   p-value
제조업 (10 ~ 33)          high_school 대한민국 - KOSPI     0.101733  0.013188
                                   독일 DAX           0.065311  0.275196
                                   영국 FTSE 100      0.087278  0.354129
                                   러셀 3000          0.071925  0.350056
                                   국고채 총수익 지수      -0.422216  0.453696
...                                                      ...       ...
보건업 및 사회복지 서비스업(86~87) university  러셀 3000         -0.016802  0.917749
                                   국고채 총수익 지수      -1.172843  0.316275
                                   국고채 콜 재투자 지수    -1.201889  0.332279
                                   국고채 제로 재투자 지수   -1.229574  0.331729
                                   KRW Cash 총수익 지수 -0.177022  0.978591

[360 rows x 2 columns]

In [17]:
eta

,대한민국 - KOSPI,독일 DAX,영국 FTSE 100,러셀 3000,국고채 총수익 지수,국고채 콜 재투자 지수,국고채 제로 재투자 지수,KRW Cash 총수익 지수
01,-0.5517,0.346257,0.183457,0.153329,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
02,0.3783,0.112757,0.128557,0.173929,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
03,0.7068,0.254557,0.118457,0.113129,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
04,-0.6121,-0.143943,-0.135443,-0.164371,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
05,0.2537,-0.296943,-0.199243,-0.208071,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
06,-0.2164,-0.546243,-0.297643,-0.318271,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
07,0.1709,0.263957,0.104457,0.207329,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
08,-0.0159,-0.033443,0.036557,0.021629,3.469447e-18,3.469447e-18,0.000000,1.734723e-18
09,0.4389,0.178257,0.128157,-0.033771,-4.671538e-02,-4.416154e-02,-0.043946,1.734723e-18
10,-0.0725,0.106957,0.068257,0.052429,1.648462e-02,1.683846e-02,0.014954,1.734723e-18


In [18]:
corr_dict = {}
for j in group_rid_dict.keys():
    
    corr_df = pd.DataFrame()
    for i in [0, 1, 2]:
        
        corr_list = []
        for k in eta.columns:
            corr_list.append(np.corrcoef(eta.loc[ksi[j].index, :][k], ksi[j][ksi[j].columns[i]])[0, 1])
        
        temp_corr_df = pd.DataFrame(corr_list, index=eta.columns)
        corr_df = pd.concat([corr_df, temp_corr_df], axis=1)
    
    corr_df.columns = ksi[j].columns
    corr_dict[j] = corr_df

In [19]:
corr_df = pd.DataFrame.from_dict({(i,j): corr_dict[i][j]
                        for i in corr_dict.keys()
                        for j in corr_dict[i].columns}, orient="index")

corr_df

대한민국 - KOSPI    독일 DAX  \
제조업 (10 ~ 33)                        high_school      0.651662  0.417717   
                                     college          0.281832  0.248406   
                                     university       0.030514  0.063937   
도매 및 소매업 (45~47)                     high_school      0.564696  0.246978   
                                     college          0.260638 -0.110578   
                                     university      -0.227683 -0.152197   
건설업 (41 ~ 42)                        high_school      0.493822 -0.015998   
                                     college          0.330779  0.125441   
                                     university       0.053661 -0.076892   
운수업(49~52)                           high_school      0.362856  0.068671   
                                     college         -0.280896 -0.148224   
                                     university       0.301071  0.348965   
교육 서비스업(85)                          high_school      0.594415  0.471723   
                                     college          0.515481  0.027411   
                                     university       0.085031  0.022039   
협회 및 단체, 수리 및 기타 개인 서비스 업(94~96)     high_school      0.389907  0.004532   
                                     college         -0.159744 -0.090991   
                                     university       0.141015  0.229688   
공공행정, 국방 및 사회보장 행정(84)               high_school     -0.060245  0.113998   
                                     college          0.198401  0.442844   
                                     university       0.581852  0.148313   
출판, 영상, 방송통 신 및 정보서비스업 (58 ~ 63)     high_school      0.110215  0.111823   
                                     college          0.040419  0.294136   
                                     university       0.082627 -0.266659   
금융 및 보험업 (64 ~ 66)                   high_school      0.214817 -0.155914   
                                     college          0.270635  0.008684   
                                     university      -0.318382  0.211289   
농업, 임업 및 어업 (01 ~ 03)                high_school      0.575245  0.346007   
                                     college          0.237771  0.101976   
                                     university       0.065413 -0.134182   
부동산업 및 임대업 (68 ~ 69)                 high_school      0.088553 -0.072301   
                                     college         -0.314350  0.104108   
                                     university      -0.471289 -0.234797   
숙박 및 음식점업 (55 ~ 56)                  high_school      0.081079 -0.200935   
                                     college          0.007101 -0.132649   
                                     university       0.105495  0.172640   
하수 · 폐기물 처리, 원료재생 및 환경복 원업 (37 ~ 39) high_school      0.264265  0.377616   
                                     college         -0.224565  0.043219   
                                     university      -0.310883 -0.230537   
전문, 과학 및 기술 서비스업 (70 ~ 73)           high_school     -0.145631  0.133579   
                                     college         -0.030945 -0.127014   
                                     university       0.058432  0.053749   
보건업 및 사회복지 서비스업(86~87)               high_school      0.161026  0.047235   
                                     college         -0.003503 -0.278745   
                                     university      -0.236457 -0.009214   

                                                  영국 FTSE 100   러셀 3000  \
제조업 (10 ~ 33)                        high_school     0.354331  0.337412   
                                     college         0.168280  0.098659   
                                     university     -0.008187 -0.032983   
도매 및 소매업 (45~47)                     high_school     0.286900  0.209488   
                                     college        -0.152891 -0.135157   
                                     university     -0.124508 -0.220456   
건설업 (41 ~ 42)               

In [20]:
corr_mat_dic = {}
for i in ksi.columns:
    corr_mat_dic[i] = pd.concat([ksi[i], eta], axis=1).dropna().corr()
    
sample_corr_mat = pd.DataFrame.from_dict({(i,k): corr_mat_dic[i][k]
                            for i in corr_mat_dic.keys()
                            for k in corr_mat_dic[i].columns}, orient="index")
sample_corr_mat

(제조업 (10 ~ 33), high_school)  \
(제조업 (10 ~ 33), high_school)         (제조업 (10 ~ 33), high_school)                      1.000000   
                                     대한민국 - KOSPI                                      0.651662   
                                     독일 DAX                                            0.417717   
                                     영국 FTSE 100                                       0.354331   
                                     러셀 3000                                           0.337412   
...                                                                                         ...   
(보건업 및 사회복지 서비스업(86~87), university) 러셀 3000                                                NaN   
                                     국고채 총수익 지수                                             NaN   
                                     국고채 콜 재투자 지수                                           NaN   
                                     국고채 제로 재투자 지수                                          NaN   
                                     KRW Cash 총수익 지수                                        NaN   

                                                                   대한민국 - KOSPI  \
(제조업 (10 ~ 33), high_school)         (제조업 (10 ~ 33), high_school)      0.651662   
                                     대한민국 - KOSPI                      1.000000   
                                     독일 DAX                            0.627000   
                                     영국 FTSE 100                       0.663290   
                                     러셀 3000                           0.555705   
...                                                                         ...   
(보건업 및 사회복지 서비스업(86~87), university) 러셀 3000                           0.555705   
                                     국고채 총수익 지수                       -0.476768   
                                     국고채 콜 재투자 지수                     -0.474082   
                                     국고채 제로 재투자 지수                    -0.471153   
                                     KRW Cash 총수익 지수                  -0.086017   

                                                                     독일 DAX  \
(제조업 (10 ~ 33), high_school)         (제조업 (10 ~ 33), high_school)  0.417717   
                                     대한민국 - KOSPI                  0.627000   
                                     독일 DAX                        1.000000   
                                     영국 FTSE 100                   0.916633   
                                     러셀 3000                       0.863210   
...                                                                     ...   
(보건업 및 사회복지 서비스업(86~87), university) 러셀 3000                       0.863210   
                                     국고채 총수익 지수                   -0.456532   
                                     국고채 콜 재투자 지수                 -0.451282   
                                     국고채 제로 재투자 지수                -0.453166   
                                     KRW Cash 총수익 지수               0.055133   

                                                                   영국 FTSE 100  \
(제조업 (10 ~ 33), high_school)         (제조업 (10 ~ 33), high_school)     0.354331   
                                     대한민국 - KOSPI                     0.663290   
                                     독일 DAX                           0.916633   
                                     영국 FTSE 100                      1.000000   
                                     러셀 3000                          0.919927   
...                                                                        ...   
(보건업 및 사회복지 서비스업(86~87), university) 러셀 3000                          0.919927   
                                     국고채 총수익 지수                      -0.560185   
                                     국고채 콜 재투자 지수                    -0.554563   
                                     국고채 제로 재투자 지수                   -0.554810   
                              

In [21]:
ksi.columns

MultiIndex([(                       '제조업 (10 ~ 33)', 'high_school'),
            (                       '제조업 (10 ~ 33)',     'college'),
            (                       '제조업 (10 ~ 33)',  'university'),
            (                    '도매 및 소매업 (45~47)', 'high_school'),
            (                    '도매 및 소매업 (45~47)',     'college'),
            (                    '도매 및 소매업 (45~47)',  'university'),
            (                       '건설업 (41 ~ 42)', 'high_school'),
            (                       '건설업 (41 ~ 42)',     'college'),
            (                       '건설업 (41 ~ 42)',  'university'),
            (                          '운수업(49~52)', 'high_school'),
            (                          '운수업(49~52)',     'college'),
            (                          '운수업(49~52)',  'university'),
            (                         '교육 서비스업(85)', 'high_school'),
            (                         '교육 서비스업(85)',     'college'),
            (                     

In [22]:
item = (                  '금융 및 보험업 (64 ~ 66)',  'university')
corr_mat_dic[item].to_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\"+str(item)+"_corr(5_index_without_nasdaq).xlsx")